In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# pull training X and y data
X_train = pd.read_csv('../input/X_test.csv')
y_train = pd.read_csv('../input/y_train.csv')
X_test = pd.read_csv('../input/X_test.csv')

In [ ]:
def MIN_MAX(frame, _columns, columns_):
    min_max_scaler = preprocessing.MinMaxScaler()
    _frame = frame[_columns]
    frame_ = frame[columns_]
    frameV = frame_.values
    frameVScaled = min_max_scaler.fit_transform(frameV)
    frameVScaledframe = pd.DataFrame(frameVScaled)
    _frameframeVScaledframe = pd.concat([_frame, frameVScaledframe], axis = 1)
    _frameframeVScaledframe.columns = frame.columns
    return _frameframeVScaledframe

In [ ]:
X_train = MIN_MAX(X_train, X_train.columns[:3], X_train.columns[3:])

In [ ]:
labels = y_train['surface'].unique()
labels

In [ ]:
def get_label_sid(label_index):
    return y_train[y_train['surface'] == labels[label_index]]['series_id'].values

In [ ]:
X_train.dtypes

In [ ]:
def construct_corrcoef_XXt(series_id, X, columns):
    return np.corrcoef(X[X['series_id'] == series_id][columns])

def construct_corrcoef_XtX(series_id, X, columns):
    return np.corrcoef(X[X['series_id'] == series_id][columns].T)

def construct_X(series_id, X, columns):
    return X[X['series_id'] == series_id][columns]

In [ ]:
cols = X_train.columns[3:]
cols

## Correleation Matrices as a basis for classifiaction
Here, we seek to construct a Convolutional Autoencoder for the constructed Correleation Matrices. The latent vectors will contribute to our loss function through the maximization of a Calinski-Harabaz Index and a Minimization of Mean Square Error with the output.

$$ \text{Loss} = \text{min}_{MSE}\text{max}_{CHI} \sum_{i} \sum_{j} (y_{ij} - y_{ij}^q)^{2} + \lambda \text{Calinski-Harabaz Index on latent vector space} $$

I'm not sure if this will work, but I think it's worth a try...

If this doesn't work, lets use the standard Varitational Autoencoder paradigm with the Calinkski-Harabaz Index as an additional regulrization term.

In [ ]:
X = np.stack([construct_X(series_id, X_train, cols) for series_id in X_train['series_id'].unique()], axis = 2)
XXt = np.stack([construct_corrcoef_XXt(series_id, X_train, cols) for series_id in X_train['series_id'].unique()], axis = 2)
XtX = np.stack([construct_corrcoef_XtX(series_id, X_train, cols) for series_id in X_train['series_id'].unique()], axis = 2)

In [ ]:
XXt.shape, XtX.shape, X.shape

In [ ]:
series_id = 21

In [ ]:
def plot_corrcoef():
    fig = plt.figure(figsize = (16, 8))
    
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan = 2)
    ax2 = plt.subplot2grid((2, 2), (1, 1))
    ax3 = plt.subplot2grid((2, 2), (1, 0))
    
    ax1.matshow(X[:,:,series_id].T)
    ax1.set_title('X')
    ax2.matshow(XXt[:,:,series_id])
    ax2.set_title('XXt')
    ax3.matshow(XtX[:,:,series_id])
    ax3.set_title('XtX')

In [ ]:
plot_corrcoef()

## Difference Tensor Structure
Look to construct a tensor whose elements are the normalized difference vectors ie.
$$ T_{i, j} = \frac{A_i - A_j^T}{|A_i - A_j|} \text{ where } i \neq j, \text{ else } \frac{A_i}{|A_i|} $$
This structure is an attempt to capture information on the nature of the change we see between data. Notice that $T_{i,j,:}$ is equivelent to the direction of change between the $i^{th}$ observation and the $j^{th}$ observation. Consider the addition of padding which represents the lengths of each of these vectors. Otherwise, this will be lost.

In [ ]:
def APS(A):
    T = np.zeros(shape = (A.shape[0], A.shape[0], A.shape[1]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i != j:
                T[i, j, :] = (A[i, :] - A.T[:, j]) / (np.linalg.norm(A[i, :] - A.T[:, j]))
            else:
                T[i, j, :] = A[i, :] / (np.linalg.norm(A[i, :]))
    return T

def APS_NN(A):
    T = np.zeros(shape = (A.shape[0], A.shape[0], A.shape[1]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i != j:
                T[i, j, :] = (A[i, :] - A.T[:, j])
            else:
                T[i, j, :] = A[i, :]
    return T

In [ ]:
TBulk = np.stack([APS(X[:,:,i]) for i in X_train['series_id'].unique()], axis = 3)

In [ ]:
T = TBulk[:,:,:,series_id]

In [ ]:
def plot_tensor_T(T_):
    fig, axes = plt.subplots(2, 5, figsize = (16, 8))
    for i in range(axes.shape[0]):
        for j in range(axes.shape[1]):
            axes[i,j].matshow(T_[:,:,axes.shape[1]*i + j])
            axes[i,j].set_title(cols[axes.shape[1]*i + j]) 

In [ ]:
plot_tensor_T(T)

In [ ]:
def pullSubIntervals(t, interval_num):
    return np.stack([t[i:(i+interval_num),i:(i+interval_num),:] for i in range(128 - interval_num)], axis = 3)

In [ ]:
pullSubIntervals(TBulk[:,:,:,series_id], 64).shape